# 名刺管理ツール

## 概要
- PDFスキャンした名刺を、OCRとLLMを使用し、Google Spread Sheetsに整理します。
- 名刺の画像は'card_images', 名刺のOCR情報は'personal_info.gsheet'に保存されます。

## 実行方法
1. ScanSnap等で名刺を両面印刷したPDFを'input_pdfs'にアップロードしてください。
1. 上記タブの"ランタイム/全て実行"をクリックしてください。

## 備考
- Google Vision APIとChatGPTを使用しているため、実行にはお金がかかります。
- ご利用の際はAPIの登録をお願いします。

© 2024 Hiroki Nakanishi


In [ ]:
# IMPORT

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

import io
import os
import glob
import pandas as pd
import json

!pip install --upgrade google-cloud-vision| tail -n -1
from google.cloud import vision

!pip install openai
from openai import OpenAI

!pip install PyPDF2
!pip install pdf2image

import PyPDF2
from pdf2image import convert_from_path

!sudo apt-get install poppler-utils
from google.colab import files
!pip install kora
from kora.xattr import get_id

import uuid
import shutil
from datetime import datetime

import time
import pandas as pd
from pandas import json_normalize
from tqdm.notebook import tqdm


# FOLDER
base_path = '/content/drive/MyDrive/mymy/Businesscard'
input_folder = f'{base_path}/input_pdfs'
tmp_folder = f'{base_path}/input_pdfs/tmp_pdf'
os.makedirs(tmp_folder, exist_ok=True)
save_folder = f'{base_path}/card_images/'
os.makedirs(save_folder, exist_ok=True)


# GET OCR API and LLM API
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f'{base_path}/api_files/your_key.json'
g_client = vision.ImageAnnotatorClient()

ai_client = OpenAI(api_key="your_key")

# SPREAD SHEET URL
ss_url = "https://docs.google.com/spreadsheets/d/1a6kBfooirvxrDV1YryjUW7TNYBq824s105dPAlH8yy0/edit#gid=1780343650"

# SPLIT PDF
input_pdfs = glob.glob(f'{input_folder}/*.pdf')

def split_pdf(input_pdf, output_prefix):
    with open(input_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for i in range(0, num_pages, 2):
            writer = PyPDF2.PdfWriter()
            for j in range(i, min(i + 2, num_pages)):
                writer.add_page(reader.pages[j])

            output_file = f"{output_prefix}_part_{i // 2 + 1}.pdf"
            with open(output_file, 'wb') as out:
                writer.write(out)

for input_pdf in input_pdfs:
    base_name = os.path.splitext(os.path.basename(input_pdf))[0]
    output_prefix = f"{tmp_folder}/biz_{base_name}_split"
    split_pdf(input_pdf, output_prefix)





# OCR and LLM EXTRACT
data = []
src_files = glob.glob(f'{tmp_folder}/*.pdf')
src_files.sort()

for file_name in tqdm(src_files):

    # OCR TWO SIDES OF BUISINESSCARD
    images = convert_from_path(file_name)
    string = ''
    for i, image in enumerate(images):
        with io.BytesIO() as output:
            image.save(output, format="JPEG")
            content = output.getvalue()

        image = vision.Image(content=content)
        response =  g_client.document_text_detection(
                image=image,
                image_context={'language_hints': ['ja']}
            )
        if response.text_annotations:
          string_page = response.text_annotations[0].description
          string += string_page + "\n\n"

    # LLM EXTRACT
    schema = {
        "会社名": "string",
        "部署名": "string",
        "氏名": "string",
        "氏名（英語）": "string",
        "会社住所": "string | null",
        "電話番号": "string | null",
        "e-mailアドレス": "string | null",
    }

    response = ai_client.chat.completions.create(
      messages=[
        {"role": "system", "content": f"次の文字列からなるべく日本語表記の会社名、日本語表記の部署名、日本語表記の氏名、英語表記の氏名、日本語表記の会社住所、電話番号、e-mailアドレスを抜き出して、JSON形式で出力してください。複数の経歴がある場合は代表的なものを会社名, 役職名として。JSONのスキーマは次の通りです : {schema}"},
        {"role": "user", "content": string}
      ],
      model="gpt-3.5-turbo",
      response_format={ "type": "json_object" }
    )
    personal_info = response.choices[0].message.content

    # SAVE PDF
    file_id = str(uuid.uuid4())  # Unique image ID
    pdf_path  = os.path.join(save_folder, f"{file_id}.pdf")
    with open(pdf_path, 'wb') as pdf_output:
        with open(file_name, 'rb') as pdf_input:
            pdf_output.write(pdf_input.read())

    # ADD EXTRA INFO
    data_row = {}
    data_row["追加日"] = datetime.now().strftime('%Y-%m-%d')  # 現在の日付を追加
    data_row["名刺画像"] = pdf_path
    data_row.update(json.loads(personal_info))

    # ADD DATA to TABLE
    data.append(data_row)

# COMPLETED TABLE
df = pd.DataFrame(data)
display(df)

# WAIT FOR PDF SAVED
time.sleep(60)

# GET DRIVE URL
def get_drive_url(file_path):
    return f'https://drive.google.com/file/d/{get_id(file_path)}/view'

df['名刺画像'] = df['名刺画像'].apply(get_drive_url)


# ADD TABLE TO SPREADSHEET
workbook = gc.open_by_url(ss_url)
worksheet = workbook.get_worksheet(0)
df.fillna('', inplace=True)
df.replace({None: ''}, inplace=True)
df_list = df.values.tolist()
headers = df.columns.tolist()
df_list.insert(0, headers)
existing_data = worksheet.get_all_values()
if len(existing_data) > 0:
    df_list = df_list[1:]
worksheet.append_rows(df_list)

# CLEAN UP
if os.path.exists(tmp_folder):
    shutil.rmtree(tmp_folder)
for pdf in input_pdfs:
    os.remove(pdf)


# DONE
display("Done!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


  0%|          | 0/77 [00:00<?, ?it/s]

,追加日,名刺画像,会社名,部署名,氏名,氏名（英語）,会社住所,電話番号,e-mailアドレス
0,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,National Cheng Kung University,Department of Biomedical Engineering,蘇芳慶,"SU, Fong-Chin","1 University Road, Tainan, Taiwan 701",+886 6 2760665,fcsu@mail.ncku.edu.tw
1,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,ETH,Bio-Inspired Robotics Lab,Dr. Fumiya Iida,Dr. Fumiya Iida,"Leonhardstrasse 27, 8092 Zurich, Switzerland",+41 44 632 07 08,Fumiya.lida@mavt.ethz.ch
2,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,Kyung Hee University,Department of Computer Engineering/ Assistant ...,辺,"Seokhee Jeon, Ph.D","1732, Deogyeong-daero, Giheung-gu, Yongin-si, ...",+82-31-201-3485,jeon@khu.ac.kr
3,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,UNIVERSITY OF EVANSVILLE,Department of Physical Therapy,Tzurei Chen,Tzurei Chen,"1800 Lincoln Avenue, Evansville, Indiana 47722",812-488-2408,tc136@evansville.edu
4,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,Liberty Mutual,RESEARCH INSTITUTE FOR SAFETY,Chien-Chi (Max) Chang,Chien-Chi (Max) Chang,"71 Frankland Road, Hopkinton, MA 01748",508.497.0260,Chien-Chi.Chang@libertymutual.com
...,...,...,...,...,...,...,...,...,...
72,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,サンタンナ先進研究学校,マイクロ・ナノ・バイオシステムおよび標的治療研究室,レオナルド・リコッティ,"Leonardo Ricotti, Ph.D.","viale Rinaldo Piaggio, 34, 56025 Pontedera (Pi...",+39 050 883074,leonardo.ricotti@santannapisa.it
73,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,"Edit Science, Inc.",Editor-in-Chief,Dee A. Worman,"Dee A. Worman, Ph.D.",None,617.628.5598,dee@editscience.com
74,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,EGYPT-JAPAN UNIVERSITY OF SCIENCE AND TECHNOLOGY,School of Innovative Design Engineering,Dr. Samy F. M. Assal,Dr. Samy F. M. Assal,"P.O.Box 179, New Borg El-Arab City, Postal Cod...",+2 0100 497 6377,samy.assal@ejust.edu.eg
75,2024-06-04,/content/drive/MyDrive/mymy/Businesscard/card_...,ASU Arizona State University,"Executive Vice President, ASU Knowledge Enterp...","Sethuraman (Panch) Panchanathan, Ph.D.","Sethuraman (Panch) Panchanathan, Ph.D.","300 East University Drive, PO Box 877205, Suit...",480-965-4087,panch@asu.edu


'Done!'